<a href="https://colab.research.google.com/github/gabrielborja/python_data_analysis/blob/main/ecuador_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ecuador Volcanoes Analysis

Ecuador is located at the center of the world, and is aslo known as the land of volcanoes. Data about Ecuador volcanoes can be found [here](https://en.wikipedia.org/wiki/List_of_volcanoes_in_Ecuador)

## Uploading packages and data

In [23]:
#Importing HTTP libraries
import requests
import pprint
from bs4 import BeautifulSoup as bs

In [ ]:
!pip freeze

In [2]:
#Importing data manipulation packages
import numpy as np
import pandas as pd

##Retrieving data from URL

In [ ]:
#class requests.Response
#Response.elapsed, Response.encoding, Response.headers, Response.json, Response.status_code, Response.text, Response.url

In [26]:
#Getting data from URL
response = requests.get('https://en.wikipedia.org/wiki/List_of_volcanoes_in_Ecuador')
print(response)
soup = bs(response.text, 'html.parser')
tables = soup.find_all('table')

<Response [200]>


In [27]:
#Printing fetched tabled lenght
len(tables)

6

In [29]:
#Print third table
pprint.pprint(tables[2])

<table class="wikitable sortable">
<tbody><tr>
<th>Name
</th>
<th>Meters
</th>
<th>Feet
</th>
<th>Coordinates
</th>
<th>Last Eruption
</th></tr>
<tr align="right">
<td align="left"><a href="/wiki/El_Altar" title="El Altar">El Altar / Kapak Urku</a></td>
<td>5405</td>
<td>17,730</td>
<td><style data-mw-deduplicate="TemplateStyles:r994658806">.mw-parser-output .geo-default,.mw-parser-output .geo-dms,.mw-parser-output .geo-dec{display:inline}.mw-parser-output .geo-nondefault,.mw-parser-output .geo-multi-punct{display:none}.mw-parser-output .longitude,.mw-parser-output .latitude{white-space:nowrap}</style><span class="plainlinks nourlexpansion"><a class="external text" href="//geohack.toolforge.org/geohack.php?pagename=List_of_volcanoes_in_Ecuador&amp;params=1.68_S_78.42_W_type:mountain" rel="nofollow"><span class="geo-nondefault"><span class="geo-dms" title="Maps, aerial photos, and other data for this location"><span class="latitude">1°41′S</span> <span class="longitude">78°25′W</span></